In [14]:
import os
import zipfile

folder_path = "C:\\Users\\Mitch\\Capstone\\Tenders"

ref_dict = {}

# for every zip
for filename in os.listdir(folder_path):
    if filename.endswith(".zip"):
        # get ref num
        ref = filename.rsplit("-specification.zip", 1)[0]
        file_path = os.path.join(folder_path, filename)

        # open read zip
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            file_list = zip_ref.namelist()
            doc_files = [file for file in file_list if file.lower().endswith(('.doc', '.docx', '.pdf'))]
            
            if doc_files:
                ref_dict[ref] = [(doc_name, file_path) for doc_name in doc_files]

for ref, doc_list in ref_dict.items():
    print(f"Reference: {ref}")
    for doc_name, file_path in doc_list:
        print(f"Document Name: {doc_name}, ZIP File Path: {file_path}")

Reference: BMW0107119
Document Name: Final Request PWST20206125.docx, ZIP File Path: C:\Users\Mitch\Capstone\Tenders\BMW0107119-specification.zip
Document Name: PWST20206125 Addendum 1.doc, ZIP File Path: C:\Users\Mitch\Capstone\Tenders\BMW0107119-specification.zip
Document Name: PWST20206125 Addendum 2.doc, ZIP File Path: C:\Users\Mitch\Capstone\Tenders\BMW0107119-specification.zip
Reference: BMW0120920
Document Name: Final Request PWST20206125.docx, ZIP File Path: C:\Users\Mitch\Capstone\Tenders\BMW0120920-specification.zip
Document Name: PWST20206125 Addendum 1.doc, ZIP File Path: C:\Users\Mitch\Capstone\Tenders\BMW0120920-specification.zip
Document Name: PWST20206125 Addendum 2.doc, ZIP File Path: C:\Users\Mitch\Capstone\Tenders\BMW0120920-specification.zip
Reference: BMW0285719
Document Name: Final Request PWST20206125.docx, ZIP File Path: C:\Users\Mitch\Capstone\Tenders\BMW0285719-specification.zip
Document Name: PWST20206125 Addendum 1.doc, ZIP File Path: C:\Users\Mitch\Capstone